# Working with timestamps

This notebook explains how to effectively work with timestamps in the Data Lake.

## Data types

First, we start with a recap of the data types involved. 

On the Data Lake side, the following types exist in Athena:

- `TIMESTAMP`: a point in time. It does not include a timezone. Note that it can have different precisions. `TIMESTAMP(3)` has millisecond precision, while `TIMESTAMP(9)` has nanoseconds precision. Keep in mind that most built-in timestamp manipulation functions don't support nanosecond (or microsecond) precision, reducing the usefulness of `TIMESTAMP(9)`.
- `TIMESTAMP WITH TIME ZONE`: same as `TIMESTAMP`, but also includes timezone information.
- `BIGINT`: a 64-bits integer.

The following integer representations of time will also be referenced:

- `unix timestamp`: the number of _seconds_ since 1st Jan 1970. It is a floating point value. Note that a unix timestamp cannot hold nanosecond precision, so converting to a unix timestamp from a `TIMESTAMP(9)` will always lead to loss of precision.
- `data lake timestamp`: the integer number of _nanoseconds_ since 1st Jan 1970.

Timestamps in the Data Lake are represented as the integer number of nanoseconds since 1st Jan 1970, and stored using the `BIGINT` data type.

## Data type serialization

In both the user interface (SQL Tools) and Python/R, the data types above will be returned in the following formats:

- `TIMESTAMP`: an integer containing the number of _milliseconds_ since 1st Jan 1970. You can multiply this value by 1e6 to get a value comparable to Data Lake timestamps stored in a `BIGINT`. If you need more than millisecond precision (e.g. you're selecting a `TIMESTAMP(9)`) you can cast the values to a `VARCHAR` as shown later in this notebook.
- `TIMESTAMP WITH TIME ZONE`: a string representation, with the following format: `YYYY-MM-dd HH:mm:ss.sssssssss Timezone`.
- `BIGINT`: a 64-bits integer.

## Converting between formats

You can see a list of available date/time functions here: https://prestodb.io/docs/current/functions/datetime.html.
For timezone information, see https://docs.aws.amazon.com/athena/latest/ug/athena-supported-time-zones.html.

In particular, the following table shows how to convert from the format in the row header to the format in the column header:

|CONVERSION|TIMESTAMP(9)|TIMESTAMP(3)|TIMESTAMP(3) WITH TIME ZONE|unix timestamp|data lake timestamp|
|-|-|-|-|-|-|
|TIMESTAMP(9)|`x`|`CAST(x AS TIMESTAMP(3))`|`AT_TIMEZONE(CAST(x AS TIMESTAMP(3)), 'US/Eastern')`|`TO_UNIXTIME(x)`|`CAST(FLOOR(TO_UNIXTIME(x)) AS BIGINT) * 1000000000 + CAST(SUBSTRING(CAST(x AS VARCHAR), 21) AS BIGINT)`|
|TIMESTAMP(3)|`CAST(x AS TIMESTAMP(9))` | `x`| `AT_TIMEZONE(x, 'US/Eastern')` | `TO_UNIXTIME(x)` | `CAST(TO_UNIXTIME(x) * 1000 AS BIGINT) * 1000000` |
|TIMESTAMP(3) WITH TIME ZONE|`CAST(FROM_UNIXTIME(TO_UNIXTIME(x)) AS TIMESTAMP(9))`|`FROM_UNIXTIME(TO_UNIXTIME(x))`| `x` | `TO_UNIXTIME(x)` | `CAST(TO_UNIXTIME(x) * 1000 AS BIGINT) * 1000000` |
|unix timestamp|`FROM_UNIXTIME_NANOS(x * 1e9)`| `FROM_UNIXTIME(x)` | `AT_TIMEZONE(FROM_UNIXTIME(x), 'US/Eastern')` | `x` | `CAST(x * 1000 AS BIGINT) * 1000000` |
|data lake timestamp|`FROM_UNIXTIME_NANOS(x)`| `FROM_UNIXTIME(ROUND(x / 1e9, 3))` | `AT_TIMEZONE(FROM_UNIXTIME(ROUND(x / 1e9, 3)), 'US/Eastern')` | `ROUND(x / 1e9, 3)` | `x`|

On top of these tables, the following operations will be useful:

- `CAST(x AS VARCHAR)`: if x is a `TIMESTAMP`, returns it as a readable string instead of as an integer.
- `x % 1000000000`: if x is a `data lake timestamp`, extracts the fractional seconds (with nanoseconds precision) part of the timestamp as an integer.
- `x + n`: if x is a `data lake timestamp` with 0 milliseconds/microseconds/nanoseconds, adds the n fractional seconds (with nanosecond precision) part to it.


## Examples

The following Python/SQL examples show some common uses of the functions above.

In [ ]:
import datetime

import maystreet_data as md
import pandas as pd
import numpy as np

Return human-readable data lake timestamps (UTC):

In [ ]:
query = """
SELECT
    CAST(FROM_UNIXTIME_NANOS(exchangetimestamp) AS VARCHAR) AS exchangetimestamp
FROM 
    p_mst_data_lake.mt_trade
WHERE 
    product = 'AAPL'
    AND f = 'xdp_nyse_integrated'
    AND dt = '2022-03-14'
LIMIT 1
"""

list(md.query(md.DataSource.DATA_LAKE, query))

Return times in a selected timezone:

In [ ]:
query = """
SELECT
    AT_TIMEZONE(FROM_UNIXTIME_NANOS(exchangetimestamp), 'US/Eastern') AS exchangetimestamp
FROM 
    p_mst_data_lake.mt_trade
WHERE 
    product = 'AAPL'
    AND f = 'xdp_nyse_integrated'
    AND dt = '2022-03-14'
LIMIT 1
"""

list(md.query(md.DataSource.DATA_LAKE, query))

Use the `datetime64` numpy type to hold data lake timestamps:

In [ ]:
query = """
SELECT
    exchangetimestamp
FROM
    p_mst_data_lake.mt_trade
WHERE 
    product = 'AAPL'
    AND f = 'xdp_nyse_integrated'
    AND dt = '2022-03-14'
LIMIT 1
"""

df = pd.DataFrame(md.query(md.DataSource.DATA_LAKE, query))

# NOTE: this is in the UTC timezone
pd.to_datetime(df['exchangetimestamp'])

Filter against data lake timestamps using user-friendly date formats, returning UTC times and price:

In [ ]:
query = """
SELECT
    CAST(FROM_UNIXTIME_NANOS(exchangetimestamp) AS VARCHAR) AS exchangetimestamp,
    price
FROM 
    p_mst_data_lake.mt_trade
WHERE 
    product = 'AAPL'
    AND f = 'xdp_nyse_integrated'
    AND dt = '2022-03-14'
    AND exchangetimestamp BETWEEN CAST(TO_UNIXTIME(TIMESTAMP '2022-03-14 15:01:03 US/Eastern') * 1000 AS BIGINT) * 1000000 + 000000001
                              AND CAST(TO_UNIXTIME(TIMESTAMP '2022-03-14 15:02:03 US/Eastern') * 1000 AS BIGINT) * 1000000 + 999999999
"""

pd.DataFrame(md.query(md.DataSource.DATA_LAKE, query))

Return human-readable data lake timestamps (UTC), splitting out the nanoseconds in a separate column:

In [ ]:
query = """
SELECT
    CAST(FROM_UNIXTIME(FLOOR(exchangetimestamp / 1e9)) AS VARCHAR) AS date_time_part,
    exchangetimestamp % 1000000000 AS seconds_fractional_part
FROM 
    p_mst_data_lake.mt_trade
WHERE 
    product = 'AAPL'
    AND f = 'xdp_nyse_integrated'
    AND dt = '2022-03-14'
LIMIT 1
"""

list(md.query(md.DataSource.DATA_LAKE, query))

Extract a Python datetime (without fractional seconds) and nanoseconds from a `data lake timestamp`:

In [ ]:
query = """
SELECT
    exchangetimestamp
FROM 
    p_mst_data_lake.mt_trade
WHERE 
    product = 'AAPL'
    AND f = 'xdp_nyse_integrated'
    AND dt = '2022-03-14'
LIMIT 1
"""

rows = list(md.query(md.DataSource.DATA_LAKE, query))

exchange_timestamp = rows[0]['exchangetimestamp']

as_datetime = datetime.datetime.utcfromtimestamp(exchange_timestamp // 1_000_000_000)
fractional_seconds = exchange_timestamp % 1_000_000_000

exchange_timestamp, as_datetime, fractional_seconds

Convert a Python datetime and fractional seconds to a `data lake timestamp`:

In [ ]:
# as_datetime = datetime.datetime(2022, 3, 14, 19, 59, 46)
# fractional_seconds = 575338496

as_datetime = datetime.datetime(2022, 3, 14, 19, 2, 3)
fractional_seconds = 999999999


data_lake_timestamp = int(as_datetime.timestamp()) * 1_000_000_000 + fractional_seconds

data_lake_timestamp, as_datetime, fractional_seconds

# Common errors

Why do I get the `SERIALIZATION_ERROR: Could not serialize column '...' of type 'timestamp(3)' at position ...` error?

This happens when returning the result of applying `FROM_UNIXTIME` to a unix timestamp not rounded to milliseconds precision.

In [ ]:
import maystreet_data as md

# WRONG: firstreceipttimestamp is not rounded
query = """
SELECT
    FROM_UNIXTIME(firstreceipttimestamp / 1e9)
FROM 
    p_mst_data_lake.mt_aggregated_price_update
WHERE 
    product = 'AAPL'
    AND f = 'xdp_nyse_integrated'
    AND dt = '2022-03-14'
LIMIT 1
"""
# print(list(md.query(md.DataSource.DATA_LAKE, query)))

# CORRECT: firstreceipttimestamp is rounded to milliseconds precision
query = """
SELECT
    FROM_UNIXTIME(ROUND(firstreceipttimestamp / 1e9, 3))
FROM 
    p_mst_data_lake.mt_aggregated_price_update
WHERE 
    product = 'AAPL'
    AND f = 'xdp_nyse_integrated'
    AND dt = '2022-03-14'
LIMIT 1
"""
print(list(md.query(md.DataSource.DATA_LAKE, query)))
